In [ ]:
import sys
from easyquant import MongoIo
from tdx_func import *
from easyquant.indicator.base import *
import pandas as pd
print("exam:python test.py <code:123456> <func-name:dqe_test_A01>")


In [ ]:
# code=sys.argv[1]
code="300059"
func="dqe_test_A01"
m=MongoIo()
data1=m.get_stock_day(code, st_start='1990-01-01',qfq=1)

# print(eval("tdx_%s" % sys.argv[2])(data))
# a=eval("tdx_%s" % func)(data)


In [ ]:
data2.head()
# data2['trade_date'] = data2.date
# data2['ts_code'] = data2.code

In [ ]:
data2=data1.reset_index()
t=10

In [ ]:
df1 = p_df[['close','high','low','date','code']].copy()
df1['cv'] = 0 #添加一列为后续保留值准备
high = df1['high']
low = df1['low']

# 保留数据的df
data = pd.DataFrame([])
#获取首行为有效的数据点,加入到保留数据框中
df1.loc[0,'cv'] = df1.iloc[0].high #最高价作为当前价
first = df1.iloc[0:1]
data = data.append(first)

#取第一个日期的最高值作为当前值,开始为0，默认为上涨周期
ci=0
cv=df1.iloc[ci].high
cup=True


In [ ]:
# print(data1.head(1))
data2

In [ ]:
data[data.period > 0].tail(30)

In [ ]:

#循环处理每一个周期
n=0
lt = t
pih = 0
pil = 0
while ci<df1.index.max():
    n=n+1
    # 取含当前日期的一个周期的最高和最低价以及索引值,如果出现下一个周期中当前点成为了这个周期的最高和最低点即当前点未变化则
    # 在前周期长度上扩展1个周期,一旦出现拐点则恢复周期。
    # 周期超高了数据长度则结束，当前点加入到数据有效点中。
    # 为什么不是从下一个点找周期，因为下一个点开始的周期则一定存在一个高低点，而这个高低点和当前点的高点或低点比较后一定会
    # 出现一个拐点，有时候不一定有拐点存在,所以要包含当前点
    ih = high[ci:ci+lt].idxmax()
    il = low[ci:ci+lt].idxmin()
    ihv = df1.iloc[ih].high
    ilv = df1.iloc[il].low
    if (ih==ci) & (il==ci):
        #数据结束了吗?如果结束了则直接添加当前数据到数据点和最后一个数据到数据点
        if (ci+lt)>df1.index.max():
            # 数据结束了,最后一个数据是否要添加到数据点中，由循环结束时处理
            break
        else:
            # 三点重叠但数据为结束 , 周期延长重新计算
            lt = lt + t
            continue
    if cup:
        # 上涨阶段
        if (ihv >= cv) & (ci != ih):
            # 如果上升周期中最高价有更新则仍然上涨持续，上涨价格有效，下跌的价格为噪声
            ci = ih
            cv = ihv
            cup = True
        else:
            # 未持续上涨，则下跌价格有效，出现了转折，此时上一个价格成为转折点价格,恢复计算周期
            df1.loc[ci,'cv'] = cv
            data = data.append(df1.iloc[ci:ci + 1])
            pil = il
            ci = il
            cv = ilv
            cup = False
            lt = t
            if abs(pih - pil) < 5:
                print("pil", ci)
    else:
        # 下跌阶段
        if (ilv<=cv) & (ci != il):
            # 下跌阶段持续创新低，则下跌价格有效，上涨价格为噪声
            ci = il
            cv = ilv
            cup = False
        else:
            # 未持续下跌，此时转为上涨，上涨价格有效，此时上一个价格成为转折点价格,恢复计算周期
            df1.loc[ci, 'cv'] = cv
            data = data.append(df1.iloc[ci:ci + 1])
            pih = ih
            ci = ih
            cv = ihv
            cup = True
            lt = t
            
            if abs(pih - pil) < 5:
                print("pih", ci)

    # print(df1.iloc[ci:ci+1])
    # print(n,ci,cv,cup,ih,il)

    # if last+t>=df1.index.max():
    #     # 最后计算恰好为最后一个周期，则直接加入最后一个周期进入数据有效点，并且结束循环
    #     last = df1.index.max()
    #     df1.loc[last, 'cv'] = df1.iloc[last].close
    #     data = data.append(df1.iloc[last:last + 1])
    #     break


In [ ]:
ci

In [ ]:
#结束了，把当前点加入到数据有效点中
df1.loc[ci, 'cv'] = cv
data = data.append(df1.iloc[ci:ci + 1])
if ci != df1.index.max():
    # 当前点不是最后一个点，则把最后一个点加入到数据点中
    df1.loc[df1.index.max(), 'cv'] = df1.iloc[df1.index.max()].close
    data = data.append(df1.tail(1))

data = data.reset_index(drop=False)
# 计算高低点转换的交易日数量即时间周期
data['period'] = (data['index'] - data['index'].shift(1)).fillna(0)
# 计算日期的差值,将字符串更改为日期
trade_date = pd.to_datetime(data['date'],format='%Y-%m-%d')
days = trade_date - trade_date.shift(1)
# 填充后转换为实际的天数数字
days = (days.fillna(pd.Timedelta(0))).apply(lambda x:x.days)
data['days'] = days
# 对日期进行转换
data['date']=trade_date.apply(lambda x:x.strftime('%Y-%m-%d'))


In [ ]:
ci=0
cv=df1.iloc[ci].high
cup=True
lt=21

In [ ]:
ih = high[ci:ci+lt].idxmax()
il = low[ci:ci+lt].idxmin()
ihv = df1.iloc[ih].high
ilv = df1.iloc[il].low


In [ ]:
cv

In [ ]:
ihv

In [ ]:
high[ci:ci+lt]

In [ ]:
high.head(3)

In [ ]:
BIDASK5VOL(data,'2021-09-15')

In [ ]:
CAPITAL(data)

In [ ]:
a.split(',')

In [ ]:
data.index[-1][0].strftime("%Y-%m-%d")

In [ ]:
a=None

In [ ]:
a


In [ ]:
mongo = MongoIo()
# return list(mongo.get_stock_list().index)


In [ ]:
data = datamongo.db['index_list'].find()

In [ ]:
data = pd.concat([data,df1.iloc[ci:ci + 1]])

In [ ]:
data1.head()

In [ ]:
df2 = data1.tail(1)

In [ ]:
data = data1.copy()

In [ ]:
pd.concat([data,df2])

In [ ]:
def h_l_line(p_df, t=21,period=1000,fn=None):
    """
    根据给定的周期找出最高最低点的日期和数据，然后计算对应的斐波纳契数据
    :param fn: 高低线输出到文件,如果文件参数为None则不输出到文件
    :param p_df:股票交易数据
    :param t:数据周期
    :param period:数据长度
    :return:有效数据点，包括股票代码，日期，高低点周期交易天数、高低点周期自然天数
    """
    if p_df is None or len(p_df)<t:
        return None
    # 获取最新的period条数据
    # df1 = p_df.tail(period).reset_index(drop=True)
    df1 = p_df[['close','high','low','trade_date','ts_code']].copy()
    df1['cv'] = 0 #添加一列为后续保留值准备
    high = df1['high']
    low = df1['low']

    # 保留数据的df
    data = pd.DataFrame([])
    #获取首行为有效的数据点,加入到保留数据框中
    df1.loc[0,'cv'] = df1.iloc[0].high #最高价作为当前价
    first = df1.iloc[0:1]
    # data = data.append(first)
    data = pd.concat([data,first])

    #取第一个日期的最高值作为当前值,开始为0，默认为上涨周期
    ci=0
    cv=df1.iloc[ci].high
    cup=True
    ptd = pd.to_datetime(df1.iloc[ci].trade_date, format='%Y-%m-%d')
    ntd = pd.to_datetime(df1.iloc[ci].trade_date, format='%Y-%m-%d')
    # print("ptd", ptd)
    #循环处理每一个周期
    n=0
    lt = t
    # while ci<df1.index.max():
    while n<df1.index.max():
        if n < ci:
            n = ci
        n=n+1
        
        # 取含当前日期的一个周期的最高和最低价以及索引值,如果出现下一个周期中当前点成为了这个周期的最高和最低点即当前点未变化则
        # 在前周期长度上扩展1个周期,一旦出现拐点则恢复周期。
        # 周期超高了数据长度则结束，当前点加入到数据有效点中。
        # 为什么不是从下一个点找周期，因为下一个点开始的周期则一定存在一个高低点，而这个高低点和当前点的高点或低点比较后一定会
        # 出现一个拐点，有时候不一定有拐点存在,所以要包含当前点
        # print("ci", ci, ci + lt,ptd,cup)
        ih = high[ci:ci+lt].idxmax()
        il = low[ci:ci+lt].idxmin()
        ihv = df1.iloc[ih].high
        ilv = df1.iloc[il].low
        # print("n1", n, ci,lt,ih,il, ihv, ilv)
        if (ih==ci) & (il==ci):
            #数据结束了吗?如果结束了则直接添加当前数据到数据点和最后一个数据到数据点
            if (ci+lt)>df1.index.max():
                # 数据结束了,最后一个数据是否要添加到数据点中，由循环结束时处理
                break
            else:
                # 三点重叠但数据为结束 , 周期延长重新计算
                lt = lt + t
                continue
        if cup:
            # 上涨阶段
            if (ihv >= cv) & (ci != ih):
                # print("n1", n, ci, cv, lt, ih, il, ihv, ilv)
                # 如果上升周期中最高价有更新则仍然上涨持续，上涨价格有效，下跌的价格为噪声
                ci = ih
                cv = ihv
                cup = True
            else:
                # 未持续上涨，则下跌价格有效，出现了转折，此时上一个价格成为转折点价格,恢复计算周期
                df1.loc[ci,'cv'] = cv
                p_ntd = ntd
                ntd = pd.to_datetime(df1.iloc[il].trade_date, format='%Y-%m-%d')
#                 print("ntd-up", ptd, p_ntd, ntd, lt, t, (ntd - ptd).days, il, ih, ilv, ihv)
                if (ntd - p_ntd).days > 0 and (ntd - ptd).days > 0 and (ntd - ptd).days < (t / 2 + 1):
                    lt = lt + int(t / 2)
                else:
                    # print("cup date", ptd,ntd)
                    ptd = ntd
                    # data = data.append(df1.iloc[ci:ci + 1])
                    data = pd.concat([data,df1.iloc[ci:ci + 1]])
                    ci = il
                    cv = ilv
                    cup = False
                    lt = t
        else:
            # 下跌阶段
            if (ilv<=cv) & (ci != il):
                # 下跌阶段持续创新低，则下跌价格有效，上涨价格为噪声
                ci = il
                cv = ilv
                cup = False
            else:
                p_ntd = ntd
                ntd = pd.to_datetime(df1.iloc[il].trade_date, format='%Y-%m-%d')
                # print("ntd-d", ptd, ntd,lt,ntd - ptd < t / 2 + 1)
#                 print("ntd-dn", ptd, p_ntd, ntd, lt, t, (ntd - ptd).days, il, ih, ilv, ihv)
                if (ntd - p_ntd).days > 0 and (ntd - ptd).days > 0 and (ntd - ptd).days < (t / 2 + 1):
                    lt = lt + int(t / 2)
                else:
                    # print("down date", ptd,ntd)
                    ptd = ntd
                    # 未持续下跌，此时转为上涨，上涨价格有效，此时上一个价格成为转折点价格,恢复计算周期
                    df1.loc[ci, 'cv'] = cv
                    # data = data.append(df1.iloc[ci:ci + 1])
                    data = pd.concat([data,df1.iloc[ci:ci + 1]])
                    ci = ih
                    cv = ihv
                    cup = True
                    lt = t

        # print(df1.iloc[ci:ci+1])
        # print(n,ci,cv,cup,ih,il)

        # if last+t>=df1.index.max():
        #     # 最后计算恰好为最后一个周期，则直接加入最后一个周期进入数据有效点，并且结束循环
        #     last = df1.index.max()
        #     df1.loc[last, 'cv'] = df1.iloc[last].close
        #     data = data.append(df1.iloc[last:last + 1])
        #     break
    #结束了，把当前点加入到数据有效点中
    print("n1", n, ci, cv, lt, ih, il, ihv, ilv, cup)
    df1.loc[ci, 'cv'] = cv
    # data = data.append(df1.iloc[ci:ci + 1])
    data = pd.concat([data,df1.iloc[ci:ci + 1]])
    if ci != df1.index.max():
        # 当前点不是最后一个点，则把最后一个点加入到数据点中
        df1.loc[df1.index.max(), 'cv'] = df1.iloc[df1.index.max()].close
        # data = data.append(df1.tail(1))
        data = pd.concat([data,df1.tail(1)])

    data = data.reset_index(drop=False)
    # 计算高低点转换的交易日数量即时间周期
    data['period'] = (data['index'] - data['index'].shift(1)).fillna(0)
    # 计算日期的差值,将字符串更改为日期
    trade_date = pd.to_datetime(data['trade_date'],format='%Y-%m-%d')
    days = trade_date - trade_date.shift(1)
    # 填充后转换为实际的天数数字
    days = (days.fillna(pd.Timedelta(0))).apply(lambda x:x.days)
    data['days'] = days
    # 对日期进行转换
    data['trade_date']=trade_date.apply(lambda x:x.strftime('%Y-%m-%d'))
    return data

In [ ]:
dfn=h_l_line(data2)

In [ ]:
data2.iloc[1046:1066]

In [ ]:
dfn

In [ ]:
import os.path
import sys
import time
from datetime import datetime, timedelta
import pandas as pd


import baostock as bs
import pandas as pd
import time
import signal

minutes_field = "date,time,code,open,high,low,close,volume,amount,adjustflag"
day_field = "date,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg,peTTM,pbMRQ,psTTM,pcfNcfTTM"
w_m_field = "date,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg"
profile_field = 'code,pubDate,statDate,roeAvg,npMargin,gpMargin,netProfit,epsTTM,MBRevenue,totalShare,liqaShare'
WEEK = 'w'
MONTH = 'm'
QFQ = '2'
HFQ = '1'
BFQ = '3'
def time_out(interval, callback):
    def decorator(func):
        def handler(signum, frame):
            raise TimeoutError("run func timeout")
  
        def wrapper(*args, **kwargs):
            try:
                signal.signal(signal.SIGALRM, handler)
                signal.alarm(interval)       # interval秒后向进程发送SIGALRM信号
                result = func(*args, **kwargs)
                signal.alarm(0)              # 函数在规定时间执行完后关闭alarm闹钟
                return result
            except TimeoutError as e:
                callback(e)
        return wrapper
    return decorator

def timeout_callback(e):
    print(e.msg)

@time_out(5, timeout_callback)
def fetch_k_day(code="sh.600606", p_begin_day: str = '2023-01-01', p_end_day: str = None):
    # 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。“分钟线”不包含指数。
    # 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg,
    # 换手率\涨跌幅（百分比）\滚动市盈率\市净率\滚动市销率\滚动市现率\
    # 日周月线指标：date,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg,peTTM,pbMRQ,psTTM,pcfNcfTTM

    # frequency：数据类型，默认为d，日k线；d = 日k线、w = 周、m = 月、5 = 5
    # 分钟、15 = 15
    # 分钟、30 = 30
    # 分钟、60 = 60
    # 分钟k线数据，不区分大小写；指数没有分钟线数据；周线每周最后一个交易日才可以获取，月线每月最后一个交易日才可以获取

    # adjustflag：复权类型，默认不复权：3；1：后复权；2：前复权。已支持分钟线、日线、周线、月线前后复权
    if p_end_day is None:
        p_end_day = time.strftime('%Y-%m-%d')
#     print(p_begin_day, p_end_day)
    rs = bs.query_history_k_data_plus(code,
                                      day_field,
                                      start_date=p_begin_day, end_date=p_end_day,
                                      frequency="d", adjustflag=QFQ)
    data_list = []
#     if rs is None:
#         return pd.DataFrame()
    while (rs.error_code == '0') & rs.next():
        # 获取一条记录，将记录合并在一起
        data_list.append(rs.get_row_data())
    result = pd.DataFrame(data_list, columns=rs.fields)
    # 添加方式
    # result.to_csv(p_file, index=False, mode='a+', header=True)
    return result


In [ ]:
lg = bs.login()
# 显示登陆返回信息
if lg.error_code != '0':
    print('login respond error_code:' + lg.error_code)
    print('login respond  error_msg:' + lg.error_msg)
    sys.exit()


In [180]:
tdata = fetch_k_day('sz.000859', p_begin_day = '2024-07-17', p_end_day = '2024-10-30')

In [182]:
tdata.iloc[-1].close

'4.0700000000'

In [183]:
type(tdata.iloc[-1].close)

str

In [184]:
float(tdata.iloc[-1].close)

4.07